In [27]:
import torch
import torch.nn as nn
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import savemat
import math
import time

In [28]:
import numpy as np
import os
import matplotlib.pyplot as plt
from glob import glob


# device = torch.device("cpu")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch_device = torch.device("cuda")
num_workers = 0

#dirs = sorted(os.listdir("outs"))
#list(enumerate(dirs))

In [29]:

ckpoint_path = "/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/epoch[00297]_tr[0.924]_va[0.917]_te[0.915]_trW[0.967]_vaW[0.936]_teW[0.917].pth.tar"
dir_save, fname_save = os.path.split(ckpoint_path)  #"/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/"

In [30]:
#from models.fishnet2 import fishdw2, fishdw, fish150
#net = fishdw(num_classes=2, norm="bn", act="lrelu")
from models.fishnet2_2d import fishdeep1_2d, fishdeep10_2d
net = fishdeep10_2d(input_c=12, num_classes=10, norm="in", act="lrelu")


if torch.cuda.device_count() > 1:
    net = nn.DataParallel(net)
state = torch.load(ckpoint_path)["network"]

first_key = next(iter(state))
if isinstance(net, nn.DataParallel):
    if first_key.startswith("module."):
        net.load_state_dict(state)
    else:
        new_dict = {"module." + key: value for key, value in state.items()}
        net.load_state_dict(new_dict)
else:
    if first_key.startswith("module."):
        new_dict = {key.replace("module.", ""): value for key, value in state.items()}
        net.load_state_dict(new_dict)
    else:
        net.load_state_dict(state)

#noet = net.module
net.to(torch_device)
net.eval()
print("Net Load")

Tail Channels :  [16, 32, 64, 128, 256, 512, 1024]
Body Channels :  [(1024, 1024), (1536, 768), (1024, 512), (640, 320), (384, 192), (224, 112)]
Head Channels :  [(128, 128), (352, 352), (736, 736), (1376, 1376), (2400, 2400), (3936, 3936), (4960, 4960)]
FishNet Initialzation Start
Net Load


In [31]:
len(new_dict.keys())

146

In [32]:
len(net.state_dict().keys())

146

In [33]:
from visualizes.GradCam2d import GradCam2d
#from visualizes.GradCam import GradCam
from visualizes.GuidedBackpropReLUModel import GuidedBackpropReLUModel

In [34]:

import datas.preprocess3d as pp3d # TRAIN_AUGS_3D, TEST_AUGS_3D, crop_shape, size_z
import datas.preprocess25d as pp25d # #TRAIN_AUGS_25D, TEST_AUGS_25D, TRAIN_AUGS_25D_v4, TEST_AUGS_25D_v4
import datas.preprocess2d as pp2d # TRAIN_AUGS_2D, TEST_AUGS_2D
from datas.TomoLoader import TomoLoader
from datas import TomoPatchLoader

size_pad = 2048
size_crop = (2048,2048,12)
rate_overlap = 4.0

pad25d = lambda img: pp25d.pad_25d(img, (size_pad, size_pad))
cencrop25d = lambda img: pp25d.center_crop_25d_alt(img, size_crop)
tform_test = [
    pad25d,
    cencrop25d,
    pp25d.calibration,
    pp25d.channel_fromz,
    pp25d.to_tensor
]

randcrop25d = lambda img: pp25d.random_crop_25d_alt(img, size_crop)
tform_train = [
    randcrop25d,
    pp25d.flipud_3d,
    pp25d.fliplr_3d,
    pp25d.swapaxes_3d,
    pp25d.calibration,
    pp25d.gaussian_3d,
    pp25d.channel_fromz,
    pp25d.to_tensor,
]
data_dir = '/workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/'
pats_exclude = ('230427',)+('230713',)+('230714',)+('230425',)+('H9',)+('h9',)+('JAX',)+('jax',)

pats_class = ('02_ecto_06h',)+('02_ecto_12',)+('02_ecto_24h',)+('03_meso_06h',)+('03_meso_12',)+('03_meso_24h',)+('04_endo_06h',)+('04_endo_12',)+('04_endo_24h',)+('05_ctl',)
mode_class = 0
reset_class = True

from datas.TomoLoader import TomoLoader
data_path = "/data02/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_testiPSC/"
# val_pdsets, classes_, class_to_idx_ = TomoPatchLoader.getdatasets(path = os.path.join(data_dir, "val"), patch_size = size_crop,
#                                             pad_size = size_pad, rate_overlap = rate_overlap,
#                                             transform = tform_test, test = True, aug_rate = 0,
#                                             pats_exclude = pats_exclude, pats_class = pats_class,
#                                             reset_class = reset_class, mode_class = mode_class)
test_pdsets, classes_, class_to_idx_ = TomoPatchLoader.getdatasets(path = os.path.join(data_dir, "test"), patch_size = size_crop,
                                            pad_size = size_pad, rate_overlap = rate_overlap,
                                            transform = tform_test, test = True, aug_rate = 0,
                                            pats_exclude = pats_exclude,pats_class = pats_class,
                                            reset_class = reset_class, mode_class = mode_class)

{'02_ecto_06h': 0, '02_ecto_12h': 1, '02_ecto_24h': 2, '03_meso_06h': 3, '03_meso_12h': 4, '03_meso_24h': 5, '04_endo_06h': 6, '04_endo_12h': 7, '04_endo_24h': 8, '05_ctl': 9}
('230427', '230713', '230714', '230425', 'H9', 'h9', 'JAX', 'jax')
Dataset Dir :  /workspace01/gkim/stem_cell_jwshin/data/23_SEC1H5_wider_v3_allh_onRA/test len :  71


In [35]:
import torch.nn.functional as F
name_part = "head_layer" # "body_layer" # "tail_layer"
idx_layer = 2
# model._modules[name_part][idx_layer]._modules['layer'][0]

# name_part = "head_layer" # "body_layer" # "tail_layer"
# idx_layer = 3
# model._modules[name_part][idx_layer]._modules['layer']._modules['layer'][0]

In [36]:
# pdset = test_pdsets[2]
# loader = torch.utils.data.DataLoader(pdset, 1, num_workers = num_workers)
# i, (input_, target_, inputDir_, coor_off_) = next(enumerate(loader))

# input_, target_ = input_.to(torch_device), target_.to(torch_device)
# output_, feat_ = net(input_)
# torch.cuda.empty_cache()
# grad_cam = GradCam2d(net, name_part, idx_layer, device=torch_device) #hooks=["features", "transition1"]
# respond_cam0, gdcam0, gdcampp0, predict = grad_cam.multicam(input_, 0)
# print(pdset.readimg_h5()[0].shape)
# print(gdcampp0.shape)
# print(len(pdset))

In [39]:
len(classes_)

In [40]:
## test set only 
for pdset in test_pdsets:
    
    grad_cam = GradCam2d(net, name_part, idx_layer, device=torch_device) #hooks=["features", "transition1"]
    
    loader = torch.utils.data.DataLoader(pdset, 1, num_workers = num_workers)
    # input_, label_ = loader.dataset.readimg_h5()
    # shape_stitch = input_.shape_
    # del input_, label_
    # grad_cam_stitch = np.zeros(shape_stitch[:2])
    # grad_cam_stitch = np.zeros(shape_stitch[:2])
    i, (input_, target_, inputDir_, coor_off_) = next(enumerate(loader))
    input_ = input_.to(torch_device)

    input_.requires_grad_(True)    
    gdcam0, idx_pred0 = grad_cam(input_, 0)
    #print(gdcam0.shape)

    gdcam_stack = np.empty((gdcam0.shape[0],gdcam0.shape[1],len(classes_),len(pdset)))
    offset_stack = pdset.patch_offset
    idx_pred_stack = np.empty((len(classes_),len(pdset)))
    del grad_cam, input_
    for i, (input_, target_, inputDir_, coor_off_) in enumerate(loader):
        
        grad_cam = GradCam2d(net, name_part, idx_layer, device=torch_device) #hooks=["features", "transition1"]
        torch.cuda.empty_cache()

        input_.requires_grad_(True)    
        input_ = input_.to(torch_device)

        #gb_model = GuidedBackpropReLUModel(model=net, device=torch_device)
        if True:#if i == math.floor(len(loader)/2):
            img = input_[0].detach().cpu().numpy()
            fig = plt.figure(figsize=(20, 20))
            ax = plt.subplot(1, 3, 1)
            ax.set_title("RI")
            plt.imshow(np.squeeze(np.max(img, axis = 0)), cmap = "gray", vmin = 0, vmax = 1)
        for idx_cls in range(len(classes_)):
            gdcam_, idx_pred_ = grad_cam(input_, 0)
            gdcam_stack[:,:,idx_cls,i] = gdcam_
            if True:#if i == math.floor(len(loader)/2) and idx_cls == target_[0]:
                ax = plt.subplot(1, 3, 2)
                ax.set_title("GradCAM - GT ({})".format(classes_[idx_cls]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)
            if True:#if i == math.floor(len(loader)/2) and idx_cls  == idx_pred_:    
                ax = plt.subplot(1, 3, 3)
                ax.set_title("GradCAM - pred ({})".format(classes_[idx_cls]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)

        del grad_cam, input_
        time.sleep(2.0)

    save_path = dir_save + '/cam/' + fname_save[:12] + '/' + name_part+ "{}".format(idx_layer) + '/'
    img_path =  dir_save + '/cam_png/'  + fname_save[:12] + '/' + name_part + "{}".format(idx_layer) + '/'

    #print(save_path + file_name)
    file_name = inputDir_[0][inputDir_[0].find("/test/") + 1:-4] + ".png"
    os.makedirs(os.path.dirname(save_path + file_name), exist_ok=True)
    os.makedirs(os.path.dirname(img_path + file_name), exist_ok=True)
    plt.show()
    plt.savefig(img_path + file_name, dpi=fig.dpi)
    
    file_name = inputDir_[0][inputDir_[0].find("/test/") +  1:-4]
    dict_data = {'gdcam_stack':gdcam_stack}#,
    #'label':label, 'pred':pred}#'guidedbp0':gb_img0,'guidedbp1':gb_img1, 
    savemat(save_path+file_name, dict_data)

    print(save_path + file_name)
    print(img_path + file_name)
    #print(save_path + file_name)
    
    plt.close()

/home/gkim/anaconda3/envs/env_RI2FL_github/lib/python3.8/site-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/cam/epoch[00297]/head_layer2/test/02_ecto_06h/240124.191247.GM25256_Ectoderm_6h.009.Group2.A1.S00
/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/cam_png/epoch[00297]/head_layer2/test/02_ecto_06h/240124.191247.GM25256_Ectoderm_6h.009.Group2.A1.S00
/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/cam/epoch[00297]/head_layer2/test/02_ecto_06h/240124.192845.GM25256_Ectoderm_6h.019.Group2.A1.S01
/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/cam_png/epoch[00297]/head_layer2/test/02_ecto_06h/240124.192845.GM25256_Ectoderm_6h.019.Group2.A1.S01
/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/cam/epoch[00297]/head_layer2/test/02_ecto_06h/240124.194634.GM25256_Ectoderm_6h.029.Group2.A1.S

KeyboardInterrupt: 

In [25]:
## test set only 
for pdset in test_pdsets:
    
    grad_cam = GradCam2d(net, name_part, idx_layer, device=torch_device) #hooks=["features", "transition1"]
    
    loader = torch.utils.data.DataLoader(pdset, 1, num_workers = num_workers)
    # input_, label_ = loader.dataset.readimg_h5()
    # shape_stitch = input_.shape_
    # del input_, label_
    # grad_cam_stitch = np.zeros(shape_stitch[:2])
    # grad_cam_stitch = np.zeros(shape_stitch[:2])
    i, (input_, target_, inputDir_, coor_off_) = next(enumerate(loader))
    input_ = input_.to(torch_device)

    input_.requires_grad_(True)    
    gdcam0, idx_pred0 = grad_cam(input_, 0)
    #print(gdcam0.shape)

    gdcam_stack = np.empty((gdcam0.shape[0],gdcam0.shape[1],len(classes_),len(pdset)))
    offset_stack = pdset.patch_offset
    idx_pred_stack = np.empty((len(classes_),len(pdset)))
    del grad_cam, input_
    for i, (input_, target_, inputDir_, coor_off_) in enumerate(loader):
        
        grad_cam = GradCam2d(net, name_part, idx_layer, device=torch_device) #hooks=["features", "transition1"]
        torch.cuda.empty_cache()

        input_.requires_grad_(True)    
        input_ = input_.to(torch_device)

        #gb_model = GuidedBackpropReLUModel(model=net, device=torch_device)
        if i == math.floor(len(loader)/2):
            img = input_[0].detach().cpu().numpy()
            fig = plt.figure(figsize=(20, 20))
            ax = plt.subplot(1, 3, 1)
            ax.set_title("RI")
            plt.imshow(np.squeeze(np.max(img, axis = 0)), cmap = "gray", vmin = 0, vmax = 1)
        for idx_cls in range(len(classes_)):
            gdcam_, idx_pred_ = grad_cam(input_, 0)
            gdcam_stack[:,:,idx_cls,i] = gdcam_
            if i == math.floor(len(loader)/2) and idx_cls == target_[0]:
                ax = plt.subplot(1, 3, 2)
                ax.set_title("GradCAM - GT ({})".format(classes_[idx_cls]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)
            if i == math.floor(len(loader)/2) and idx_cls  == idx_pred_:    
                ax = plt.subplot(1, 3, 3)
                ax.set_title("GradCAM - pred ({})".format(classes_[idx_cls]))
                plt.imshow(gdcam_, cmap = "turbo")#gdcam[:, :, zdim],       cmap=cmap)

        del grad_cam, input_

    save_path = dir_save + '/cam/' + fname_save[:12] + '/' + name_part+ "{}".format(idx_layer) + '/'
    img_path =  dir_save + '/cam_png/'  + fname_save[:12] + '/' + name_part + "{}".format(idx_layer) + '/'

    #print(save_path + file_name)
    file_name = inputDir_[0][inputDir_[0].find("/test/") + 1:-4] + ".png"
    os.makedirs(os.path.dirname(save_path + file_name), exist_ok=True)
    os.makedirs(os.path.dirname(img_path + file_name), exist_ok=True)
    plt.show()
    plt.savefig(img_path + file_name, dpi=fig.dpi)
    
    file_name = inputDir_[0][inputDir_[0].find("/test/") +  1:-4]
    dict_data = {'gdcam_stack':gdcam_stack}#,
    #'label':label, 'pred':pred}#'guidedbp0':gb_img0,'guidedbp1':gb_img1, 
    savemat(save_path+file_name, dict_data)

    print(save_path + file_name)
    print(img_path + file_name)
    #print(save_path + file_name)
    
    plt.close()

/tmp/ipykernel_1402097/2346335049.py:59: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/cam/epoch[00297]/head_layer2/test/02_ecto_06h/240124.191247.GM25256_Ectoderm_6h.009.Group2.A1.S00
/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/cam_png/epoch[00297]/head_layer2/test/02_ecto_06h/240124.191247.GM25256_Ectoderm_6h.009.Group2.A1.S00
/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/cam/epoch[00297]/head_layer2/test/02_ecto_06h/240124.192845.GM25256_Ectoderm_6h.019.Group2.A1.S01
/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/cam_png/epoch[00297]/head_layer2/test/02_ecto_06h/240124.192845.GM25256_Ectoderm_6h.019.Group2.A1.S01
/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/cam/epoch[00297]/head_layer2/test/02_ecto_06h/240124.194634.GM25256_Ectoderm_6h.029.Group2.A1.S

In [26]:
dir_save + '/cam/' + fname_save[:12] + '/' + name_part+ "{}".format(idx_layer) 

'/data02/gkim/stem_cell_jwshin/outs/23_SEC1H5_wider_v3_pick3h0_GM_germline_fishdeep10_b012_in_lr0.001/cam/epoch[00297]/head_layer2'